# 05 - Paramétrisation spatiale du xi

Dans ce notebook, nous allons créer un modèle spatial pour le paramètre xi des lois GEV.

In [1]:
using CSV, DataFrames, StatsBase, Random

using Distributions, Extremes, Optim

using Mamba, ForwardDiff, ProgressMeter

using IDF

import Plots

In [2]:
PROVINCES = ["NB", "NL", "NS", "ON", "PE", "QC"]#provinces considerees

DURATION = "24 h"

"24 h"

In [3]:
station_list = load_data("station_list")

filter!(row -> row.Province ∈ PROVINCES , station_list)#on ne selectionne que les stations qui nous interessent

first(station_list, 10)

,Name,Province,ID,Lat,Lon,Elevation
,String,String,String,Float64,Float64,Int64
1,BEECHWOOD,NB,8100512,46.53,-67.67,91
2,BELLEDUNE,NB,8100514,47.9,-65.83,7
3,BOUCTOUCHE CDA CS,NB,8100593,46.43,-64.77,35
4,CHARLO AUTO,NB,8100885,47.98,-66.33,42
5,MIRAMICHI RCS,NB,8100989,47.02,-65.47,33
6,EDMUNDSTON,NB,8101303,47.42,-68.32,154
7,FREDERICTON A,NB,8101500,45.87,-66.53,20
8,FREDERICTON CDA CS,NB,8101605,45.92,-66.62,35
9,MONCTON INTL A,NB,8103201,46.12,-64.68,70


In [8]:
Y = Vector{Float64}[]

# for i=1:size(station_list,1)
for stationID in station_list[:,:ID]
#     stationID = station_list[i,:ID]
    y = IDF.pcp(stationID, DURATION)
    push!(Y,y)
end

first(Y, 5)

5-element Vector{Vector{Float64}}:
 [118.6, 45.2, 69.3, 56.4, 46.0, 43.9, 74.2, 50.3, 54.6, 92.2]
 [48.8, 50.5, 54.1, 38.9, 47.0, 55.1, 45.0, 23.0, 45.0, 55.1, 85.6, 28.4, 50.6, 41.5, 35.8, 48.2, 37.8, 38.9]
 [45.0, 55.4, 65.1, 58.3, 38.0, 61.8, 38.2, 95.8, 62.8, 71.0, 67.4, 64.0, 77.2, 64.2, 59.4, 39.6, 64.0]
 [55.4, 44.4, 54.4, 74.4, 54.9, 32.0, 55.4, 46.7, 53.3, 46.7  …  49.6, 103.8, 50.4, 62.0, 57.2, 58.4, 60.8, 75.0, 46.6, 49.2]
 [25.1, 34.3, 42.9, 45.0, 54.6, 72.9, 53.6, 40.9, 41.1, 48.3  …  57.8, 33.0, 68.0, 78.0, 49.2, 61.8, 52.8, 80.6, 118.2, 147.0]

Chargeons la grille des précipitations moyennes :

In [5]:
crcm = IDF.load_data("CROQmeanPcp")

lat = reshape(crcm[:,1],300,300)
lon = reshape(crcm[:,2],300,300)
pcp = reshape(crcm[:,3],300,300)

latlim = [60, 253];
lonlim = [20, 270];

lat = lat[latlim[1]:latlim[2],lonlim[1]:lonlim[2]]
lon = lon[latlim[1]:latlim[2],lonlim[1]:lonlim[2]]
pcp = pcp[latlim[1]:latlim[2],lonlim[1]:lonlim[2]]

m₁, m₂ = size(lat)
m = m₁*m₂

griddedCovariate = DataFrame(Lat = vec(lat), Lon = vec(lon), meanPcp = vec(pcp))
first(griddedCovariate,5)

,Lat,Lon,meanPcp
,Float64,Float64,Float64
1,42.4558,-94.9948,2.41792
2,42.5658,-94.9909,2.42233
3,42.6758,-94.987,2.41666
4,42.7857,-94.9832,2.46305
5,42.8957,-94.9792,2.47048
